In [2]:
import pandas as pd
import numpy as np
import time
import math

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [6]:
training = pd.read_csv('../../../data/training.csv', parse_dates=[0])
validation = pd.read_csv('../../../data/validation.csv', parse_dates=[0])
testing = pd.read_csv('../../../data/testing.csv', parse_dates=[0])

In [7]:
training

,Unnamed: 0,Time (UTC+10),Regions VIC Trading Price ($/MWh),Regions VIC Trading Total Intermittent Generation (MW),Regions VIC Operational Demand (MW),Time,Hour,Day
0,0,2018-01-01 00:00:00,90.43,146.87,4599,00:00:00,0,1
1,1,2018-01-01 00:30:00,92.46,131.68,4398,00:30:00,0,1
2,2,2018-01-01 01:00:00,87.62,119.98,4238,01:00:00,1,1
3,3,2018-01-01 01:30:00,73.08,123.86,4112,01:30:00,1,1
4,4,2018-01-01 02:00:00,70.18,132.72,3956,02:00:00,2,1
...,...,...,...,...,...,...,...,...
52603,52603,2020-12-31 21:30:00,35.05,295.14,4421,21:30:00,21,4
52604,52604,2020-12-31 22:00:00,35.49,272.81,4353,22:00:00,22,4
52605,52605,2020-12-31 22:30:00,48.25,221.45,4323,22:30:00,22,4
52606,52606,2020-12-31 23:00:00,38.66,189.40,4330,23:00:00,23,4


In [15]:
BATTERY_POWER = 300
BATTERY_CAPACITY = 580
EFFICIENCY = 0.9
CAPACITY = 580
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
#FIXED_OP =8.1
#VAR_OP = 0

TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

In [9]:
# percentile.exc from excel != np.percentile (np.percentile == percentile.inc from excel)
# code taken from https://stackoverflow.com/questions/38596100/python-equivalent-of-excels-percentile-exc

def quantile_exc(ser, q):
    ser_sorted = ser.sort_values()
    rank = q * (len(ser) + 1) - 1
    assert rank > 0, 'quantile is too small'
    rank_l = int(rank)
    return ser_sorted.iat[rank_l] + (ser_sorted.iat[rank_l + 1] - ser_sorted.iat[rank_l]) * (rank - rank_l)

In [10]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    
    df['charge_forecast'] = 0
    df['discharge_forecast'] = 0
    
    df = df.drop([0], axis=0) 
    
    return df

In [11]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = df.iloc[i:i+LOOKAHEAD][PRICE]
            ser = pd.Series(thelist)


            if (df.at[i,PRICE] <= quantile_exc(ser,CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= quantile_exc(ser,DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1
                
        else:
            thelist = df.iloc[i:][PRICE]
            ser = pd.Series(thelist)

            if (df.at[i,PRICE] <= ser.quantile(CHARGING_PERCENTILE)):
                df.at[i,'charge_forecast'] = 1

            if (df.at[i,PRICE] >= ser.quantile(DISCHARGING_PERCENTILE)):
                df.at[i,'discharge_forecast'] = 1

    return df

In [12]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """
    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap



def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch



def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
    
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = round(price * dispatch * factor)
    return revenue



def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if(df.at[i,'charge_forecast'] == 1):
        raw_power = -min(BATTERY_POWER, (BATTERY_CAPACITY-opening_cap)/EFFICIENCY*2)
    if(df.at[i,'discharge_forecast'] == 1):
        raw_power = min(BATTERY_POWER, opening_cap/EFFICIENCY*2)
            
    return raw_power



def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df



def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

In [13]:
def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    # Find Optimal Charging and Discharging period
    df3 = algorithm3(df2)
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [24]:
# Highest Revenue using cross validation
LOOKAHEAD = 10
CHARGING_PERCENTILE = 0.32
DISCHARGING_PERCENTILE = 0.74
training_set = run_all(training)

Total revenue in the dataset: 100013456
Total days in the dataset: 1095.9791666666667
Revenue per day: 91254.88790465146
Time Complexity for running the entire Algorithm 3: 20.949450969696045s


In [25]:
training_set

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
1,2018-01-01 00:30:00,92.46,0,0,0,0,0,0,1
2,2018-01-01 01:00:00,87.62,0,0,0,0,0,0,1
3,2018-01-01 01:30:00,73.08,0,0,0,0,0,0,1
4,2018-01-01 02:00:00,70.18,0,0,0,0,0,0,1
5,2018-01-01 02:30:00,67.43,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
52603,2020-12-31 21:30:00,35.05,0,0,0,580,580,1,0
52604,2020-12-31 22:00:00,35.49,0,0,0,580,580,1,0
52605,2020-12-31 22:30:00,48.25,300,135,6455,580,430,0,1
52606,2020-12-31 23:00:00,38.66,-300,-150,-5852,430,565,1,0


In [26]:
thetime = training_set.loc[(training_set['Time (UTC+10)'] >= '2020-07-17') & (training_set['Time (UTC+10)'] < '2020-07-18')]

thetime

,Time (UTC+10),Regions VIC Trading Price ($/MWh),raw_power,dispatch,revenue,opening,closing,charge_forecast,discharge_forecast
44544,2020-07-17 00:00:00,79.09,0,0,0,0,0,0,1
44545,2020-07-17 00:30:00,75.51,0,0,0,0,0,0,1
44546,2020-07-17 01:00:00,73.98,0,0,0,0,0,0,0
44547,2020-07-17 01:30:00,75.57,0,0,0,0,0,0,1
44548,2020-07-17 02:00:00,71.94,0,0,0,0,0,0,0
44549,2020-07-17 02:30:00,74.10,0,0,0,0,0,0,0
44550,2020-07-17 03:00:00,67.36,0,0,0,0,0,0,0
44551,2020-07-17 03:30:00,58.04,-300,-150,-8785,0,135,1,0
44552,2020-07-17 04:00:00,51.85,-300,-150,-7848,135,270,1,0
44553,2020-07-17 04:30:00,74.53,0,0,0,270,270,0,0


In [27]:
sum(thetime['revenue'])

49382

In [17]:
validation_set = run_all(validation)

Total revenue in the dataset: 16578567
Total days in the dataset: 181.97916666666666
Revenue per day: 91101.45575271895
Time Complexity for running the entire Algorithm 3: 3.3710989952087402s


In [18]:
test_set = run_all(testing)

Total revenue in the dataset: 5757589
Total days in the dataset: 45.0
Revenue per day: 127946.42222222222
Time Complexity for running the entire Algorithm 3: 0.8516731262207031s


In [1]:
100013456 + 16578567

116592023